In [3]:
import pandas as pd

def compare_metrics(csv_file1, csv_file2):
    # Read CSV files into DataFrames
    df1 = pd.read_csv(csv_file1)
    df2 = pd.read_csv(csv_file2)
    
    # Extract image names from image2_path column
    df1['image_name'] = df1['image2_path'].apply(lambda x: x.split('/')[-1])
    df2['image_name'] = df2['image2_path'].apply(lambda x: x.split('/')[-1])
    
    # List of metrics to compare
    metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'specificity', 'similarity_percentage']
    
    # Initialize DataFrame for comparison result
    comparison_result = pd.DataFrame(columns=['image_name'])
    comparison_result['image_name'] = df1['image_name']
    
    # Compare each metric
    for metric in metrics:
        # Merge to compare metric values
        merged_df = pd.merge(df1[['image_name', metric]], df2[['image_name', metric]], on='image_name', suffixes=('_csv1', '_csv2'))
        
        # Compare metrics
        comparison_result[f'better_{metric}'] = merged_df.apply(lambda row: 'csv1' if row[f'{metric}_csv1'] > row[f'{metric}_csv2'] else ('csv2' if row[f'{metric}_csv2'] > row[f'{metric}_csv1'] else "Equal"), axis=1)
    
    # Calculate the ratio of times csv1 got better results to csv2 for accuracy
    better_counts_acc = comparison_result['better_accuracy'].value_counts()
    csv1_count_acc = better_counts_acc['csv1'] if 'csv1' in better_counts_acc else 0
    csv2_count_acc = better_counts_acc['csv2'] if 'csv2' in better_counts_acc else 0
    ratio_acc = csv1_count_acc / csv2_count_acc if csv2_count_acc != 0 else float('inf')
    
    # Calculate the ratio of times csv1 got better results to csv2 for other metrics
    ratio_metrics = {}
    for metric in metrics:
        better_counts = comparison_result[f'better_{metric}'].value_counts()
        csv1_count = better_counts['csv1'] if 'csv1' in better_counts else 0
        csv2_count = better_counts['csv2'] if 'csv2' in better_counts else 0
        ratio = csv1_count / csv2_count if csv2_count != 0 else float('inf')
        ratio_metrics[metric] = f"{csv1_count}/{csv2_count} ({ratio:.2f})"
    
    # Create the final row
    final_row = pd.DataFrame({
        'image_name': ['csv1/csv2'],
        'better_accuracy': [f"{csv1_count_acc}/{csv2_count_acc} ({ratio_acc:.2f})"]
    })
    for metric in metrics:
        final_row[f'better_{metric}'] = [ratio_metrics[metric]]
    
    # Concatenate final row to comparison result
    comparison_result = pd.concat([comparison_result, final_row], ignore_index=True)
    
    return comparison_result

# Example usage:
if __name__ == "__main__":
    # Paths to the CSV files
    csv_file1 = "/argusdata/users/undergrads/ItamarAndDafna/metrics_2d/metrics_Dataset030_1030GlyRM1_4_folds_500epochs.csv"
    csv_file2 = "/argusdata/users/undergrads/ItamarAndDafna/metrics_2d/metrics_Dataset030_1030GlyRM1_4_folds_250epochs.csv"
    
    # Compare metrics
    comparison_result = compare_metrics(csv_file1, csv_file2)
    
    # Display comparison result
    print("Comparison Result:")
    print(comparison_result.to_string(index=False))


Comparison Result:
   image_name better_accuracy better_precision better_recall better_f1_score better_specificity better_similarity_percentage
image_001.tif            csv1             csv1          csv1            csv1               csv1                         csv1
image_002.tif            csv1             csv2          csv1            csv1               csv2                         csv1
image_003.tif            csv2             csv2         Equal            csv2               csv2                         csv2
image_004.tif            csv1             csv2          csv1            csv1               csv2                         csv1
image_005.tif            csv1             csv1          csv1            csv1               csv1                         csv1
image_006.tif            csv2             csv1          csv2            csv2               csv1                         csv2
image_007.tif            csv1             csv1          csv1            csv1               csv1           

In [ ]:
"/argusdata/users/undergrads/ItamarAndDafna/metrics_2d/metrics_Dataset030_1030GlyRM1_500epochs.csv"
"/argusdata/users/undergrads/ItamarAndDafna/metrics_2d/metrics_Dataset030_1030GlyRM1_250epochs.csv"
"/argusdata/users/undergrads/ItamarAndDafna/metrics_2d/metrics_Dataset030_1030GlyRM1_4_folds_250epochs.csv"